## Importing Requierd Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import shutil

## Creating Train, Test and Validation Datasets

In [ ]:
# defining input data paths
base_dir = "/kaggle/input/face-mask-detection/data"
mask_path = os.path.join(base_dir, "WithMask")
without_mask_path = os.path.join(base_dir, "WithoutMask")

mask_path, without_mask_path

In [ ]:
# getting images name in each folder
mask_filenames = os.listdir(mask_path)
without_mask_filenames = os.listdir(without_mask_path)

In [ ]:
# total images
print(f"Total images in mask folder: {len(mask_filenames)}")
print(f"Total images in without mask folder: {len(without_mask_filenames)}")

We have nearly 6K images in both folders, making this dataset balanced

In [ ]:
# only taking images if they are in correct format
[file for file in mask_filenames[:5] if file.split(".")[-1] in ["jpg", "png", "jpeg"]]

In [ ]:
# shuffling the images in order to reduce bias
np.random.seed(42)
np.random.shuffle(mask_filenames)

In [ ]:
mask_filenames[:5]

In [ ]:
data_split = "data_split"
cat_name = "WithMask"
train_path = os.path.join(data_split, "train")
os.makedirs(os.path.join(train_path, cat_name), exist_ok = True)


In [ ]:
train_path

In [ ]:
# function to create train, validation and test set from given input data
def make_datasets(cat_folder_path, test_size = 0.15, val_size = 0.15, seed = 42):
  
  # only taking images if they are in correct format
  file_name = [file for file in os.listdir(cat_folder_path)
                if file.split(".")[-1].lower() in ["jpg", "png", "jpeg"] ]
  
  # shuffle file_name
  np.random.seed(seed)
  np.random.shuffle(file_name)
  train_len = int(len(file_name) *(1 - (test_size + val_size)))
  val_len = int(len(file_name)* val_size)

  # create train validation and test filenames
  train_file_names = file_name[:train_len]
  val_file_names = file_name[train_len : train_len+val_len]
  test_file_names = file_name[train_len+val_len:]

  data_split = "data_split"
  train_path = os.path.join(data_split, "train")
  val_path = os.path.join(data_split, "val")
  test_path = os.path.join(data_split, "test")
  cat_name = os.path.split(cat_folder_path)[-1]

  os.makedirs(os.path.join(train_path, cat_name), exist_ok = True)
  os.makedirs(os.path.join(val_path, cat_name), exist_ok = True)
  os.makedirs(os.path.join(test_path, cat_name), exist_ok = True)

  for file in train_file_names:
    shutil.copy(
        src = os.path.join(cat_folder_path, file),
        dst = os.path.join(train_path, cat_name, file)
    )

  for file in val_file_names:
    shutil.copy(
        src = os.path.join(cat_folder_path, file),
        dst = os.path.join(val_path, cat_name, file)
    )

  for file in test_file_names:
    shutil.copy(
        src = os.path.join(cat_folder_path, file),
        dst = os.path.join(test_path, cat_name, file)
    )
  print(f"For category -: {cat_name}")
  print(f"Train: {len(train_file_names)}")
  print(f"Val: {len(val_file_names)}")
  print(f"Test: {len(test_file_names)}")
  print("------"*10)

In [ ]:
for cat in os.listdir(base_dir):
  make_datasets(os.path.join(base_dir, cat))

## Figuring out common height and width of Images


For CNN task to perform good, we need to maintain the dimensions of the input images consistent so that model can trained properly and predictions are consistant.

In [ ]:
train_path = os.path.join("data_split", "train")
val_path = os.path.join("data_split", "val")
test_path = os.path.join("data_split", "test")

In [ ]:
# getting height and width for each image in training dataset
height = []
width = []
for folder in os.listdir(train_path):
  for img_name in os.listdir(os.path.join(train_path, folder)):
    img = cv2.imread(os.path.join(train_path, folder, img_name))
    height.append(img.shape[0])
    width.append(img.shape[1])

In [ ]:
pd.Series(height).describe()

In [ ]:
pd.Series(width).describe()

Both height and width has mean of 152 and similar distribution, which proves that all the images in the test set are in square shape.
We'll take mean of these height and width as input shape of our images as we want minimum distortion in the image while reshaping them.

## Creating Dataset Generators

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory

print("creating generator for train dataset")
try:
    train_ds = image_dataset_from_directory(
        directory=train_path,
        labels='inferred',
        label_mode='int',
        batch_size=32,
        image_size=(150, 150),
        shuffle=True,
        seed=42,
        verbose=True
    )
except Exception as e:
    print(f"Error creating train dataset: {e}")

print("creating generator for validation dataset")
valid_ds = image_dataset_from_directory(
    directory = val_path,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    shuffle = False
)

print("creating generator for train dataset")
test_ds = image_dataset_from_directory(
    directory = test_path,
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150, 150),
    shuffle = False
)

let's check how data generator outputs a batch

In [ ]:
for img, label in train_ds.take(1):
  print(img)
  print(label)

In [ ]:
# class names from data generator
train_ds.class_names

## Create a CNN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Input

# model build
model = Sequential([
    Input((150,150,3)),
    Conv2D(filters = 10, kernel_size = 3, activation = "relu"),
    Conv2D(filters = 10, kernel_size = 3, activation = "relu"),
    MaxPool2D(),
    Conv2D(filters = 10, kernel_size = 3, activation = "relu"),
    Conv2D(filters = 10, kernel_size = 3, activation = "relu"),
    MaxPool2D(),
    Flatten(),
    Dense(1, activation = "sigmoid")

])


In [ ]:
# model compile
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [ ]:
model.summary()

## Training Model

In [ ]:
#model fit
history = model.fit(
    train_ds,
    epochs = 5,
    validation_data = valid_ds
)

In [ ]:
model.evaluate(test_ds)

## Saving Model

In [ ]:
# saviing the architecture as a JSON file
model_json = model.to_json()
with open("face_detection_architecture.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# saving model
model.save('face_mask_detection.keras')